In [ ]:
#  1. IMPORTS & INITIAL SETUP
import torch
import pandas as pd
from datasets import Dataset
from peft import LoraConfig
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM
from trl import SFTTrainer

import matplotlib.pyplot as plt
import pandas as pd

# --- Configuration ---
# The base language model to be fine-tuned.
BASE_MODEL_ID = "frhew/sigdial_ft_a2"
# The directory where the final LoRA adapters will be saved.
SFT_MODEL_OUTPUT_DIR = "my_sft_tuned_model_v1"


In [ ]:
#  2. LOAD AND PREPARE THE DATASET

# --- Configuration for saved datasets ---
MASTER_TRAIN_DATASET = "sft_train_dataset.csv"
MASTER_EVAL_DATASET = "sft_eval_dataset.csv"
MASTER_TEST_DATASET = "sft_test_dataset.csv"

# --- Load and Clean Data ---
df = pd.read_csv("data/ordered_simplifications_with_rules_clean.csv", index_col=0)
df.rename(columns={"original_sentence": "original", "final_simplification": "simplified"}, inplace=True)
faulty_indices = df[
    df["applied_rules"].str.contains("convert_word_to_number") &
    df["simplified"].str.match(r"^\d+$")
].index
df = df.drop(faulty_indices)
print(f"Loaded and cleaned {len(df)} rows of data.")

# --- Create Train/Validation/Test Split (80:10:10) ---
# First, convert the entire DataFrame into a Hugging Face Dataset object.
full_dataset = Dataset.from_pandas(df)

# Step 1: Split the data into a training set (80%) and a temporary set (20%).
train_and_temp_split = full_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_and_temp_split['train']
temp_dataset = train_and_temp_split['test'] # This is the 20%

# Step 2: Split the temporary set in half to get validation (10%) and test (10%).
eval_and_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
eval_dataset = eval_and_test_split['train']
test_dataset = eval_and_test_split['test']

print(f"\nData split into {len(train_dataset)} training, {len(eval_dataset)} validation, and {len(test_dataset)} test examples.")

# --- Format for SFT ---
def format_sft_example(example):
    """Formats an example with the instruction template for the SFTTrainer."""
    return {
        "text": f"Aufgabe: Vereinfache den folgenden Satz nach Leichter-Sprache-Regeln.\nSatz: {example['original']}\nAntwort: {example['simplified']}"
    }

# Apply the formatting function to all three splits.
train_dataset = train_dataset.map(format_sft_example)
eval_dataset = eval_dataset.map(format_sft_example)
test_dataset = test_dataset.map(format_sft_example)

# --- Save all Three Datasets to Disk for Later Use ---
train_dataset.save_to_disk(MASTER_TRAIN_DATASET)
eval_dataset.save_to_disk(MASTER_EVAL_DATASET)
test_dataset.save_to_disk(MASTER_TEST_DATASET)
print("\nTraining, validation, and test datasets have been saved to disk.")

In [ ]:

# ==============================================================================
#  3. CONFIGURE MODEL, TOKENIZER, AND LORA
# ==============================================================================

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, padding_side='left')
# Set the padding token to be the end-of-sequence token if it's not already defined.
# This is a standard practice for decoder-only models like Llama.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- PEFT Configuration (LoRA) ---
# This configuration tells the trainer to use LoRA for efficient fine-tuning.
# It freezes the base model and only trains the small adapter layers.
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # Target the attention layers for LoRA adaptation.
    # Note: These module names can vary between models. "q_proj" and "v_proj" are common for Llama.
    target_modules=["q_proj", "v_proj"] 
)


In [ ]:
# ==============================================================================
#  4. CONFIGURE AND RUN THE SFTTRAINER
# ==============================================================================

# --- Load the Base Model with the Correct Precision ---
# We load the model here first, specifying bfloat16 for memory and performance gainsfor a Mac.
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# --- Define Training Arguments ---
training_args = TrainingArguments(
    output_dir=SFT_MODEL_OUTPUT_DIR,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    # Set the strategies to 'steps'
    logging_strategy="steps",
    evaluation_strategy="steps",
    # Define the interval 'N'
    logging_steps=100,  # Print training loss every 100 steps
    eval_steps=100,     # Run evaluation and print validation loss every 100 steps
    # Also align saving strategy if you want to save the best model based on steps
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss", # Use eval_loss to find the best model
    save_total_limit=2, # Optional: only keep the best 2 checkpoints
)

# --- Initialize the SFTTrainer ---
# Now, pass the pre-loaded model object instead of the model ID string.
trainer = SFTTrainer(
    model=model,  # <-- Pass the loaded model object here
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
    peft_config=peft_config,
    args=training_args,
    dataset_text_field="text",
    packing=False,
    max_seq_length=256,
)

In [ ]:

# --- Start Training ---
print("\n--- Starting Supervised Fine-Tuning (SFT)... ---")
trainer.train()
print("--- SFT Training Finished. ---")

# --- Save the Final Model ---
# This saves the trained LoRA adapter weights to the specified directory.
trainer.model.save_pretrained(SFT_MODEL_OUTPUT_DIR)
tokenizer.save_pretrained(SFT_MODEL_OUTPUT_DIR)
print(f"\nSFT model adapters and tokenizer saved to: {SFT_MODEL_OUTPUT_DIR}")

In [ ]:
# ==============================================================================
#  5. PLOT TRAINING & VALIDATION LOSS CURVE
# ==============================================================================

# --- Extract the training history ---
log_history = trainer.state.log_history
log_df = pd.DataFrame(log_history)

train_logs = log_df[log_df.get('loss').notna()]
eval_logs = log_df[log_df.get('eval_loss').notna()]

# --- Create the Plot ---
if not train_logs.empty and not eval_logs.empty:
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(train_logs['step'], train_logs['loss'], marker='o', linestyle='-', markersize=4, label='Training Loss')
    ax.plot(eval_logs['step'], eval_logs['eval_loss'], marker='s', linestyle='--', markersize=4, label='Validation Loss')

    ax.set_title('SFT Training & Validation Loss Curve', fontsize=16)
    ax.set_xlabel('Training Steps', fontsize=12)
    ax.set_ylabel('Loss', fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(True)
    
    plt.savefig("sft_loss_curve.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("\nCould not generate plot. Check if training and evaluation logs were created.")